## Relationships

### Defining a one-to-many relashionship

This shows how to define a 0TM relationship, proving a way to instanciate object with `parent=obj`, `parent_id=obj.id`, and nested init.

> It's not that easy, because it seems to have problems with `MappedAsDataclass` as described [here](https://github.com/sqlalchemy/sqlalchemy/discussions/9280).

In [1]:
from __future__ import annotations
from typing import List

from sqlalchemy import ForeignKey, create_engine
from sqlalchemy.orm import (
    Mapped,
    mapped_column,
    DeclarativeBase,
    relationship,
    sessionmaker,
)


class Base(DeclarativeBase):
    pass


class Parent(Base):
    __tablename__ = "parent_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    children: Mapped[List[Child]] = relationship(back_populates="parent")


class Child(Base):
    __tablename__ = "child_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    parent_id: Mapped[int] = mapped_column(ForeignKey("parent_table.id"))
    parent: Mapped[Parent] = relationship(back_populates="children")


engine = create_engine("sqlite://", echo=False)
session_factory = sessionmaker(bind=engine)

with session_factory() as session:
    Base.metadata.create_all(engine)

    # ------------------------------------
    # First test using plain objects
    # ------------------------------------

    p1 = Parent()
    c1 = Child(parent=p1)

    session.add_all([p1, c1])
    session.commit()

    assert c1.parent == p1
    assert c1.parent_id == p1.id
    assert p1.children == [c1]

    # ------------------------------------
    # Second test using object ids
    # ------------------------------------

    p2 = Parent()
    session.add(p2)
    session.commit()

    c2 = Child(parent_id=p2.id)

    session.add(c2)
    session.commit()

    assert c2.parent == p2
    assert c2.parent_id == p2.id
    assert p2.children == [c2]

    # ------------------------------------
    # Third test with nested init
    # ------------------------------------

    c3 = Child(parent=Parent())
    session.add(c3)
    session.commit()

    assert c3.parent_id == c3.parent.id
    assert c3.parent.children == [c3]

### Defining a many-to-many relashionship

In [2]:
from __future__ import annotations
from typing import List

from sqlalchemy import create_engine
from sqlalchemy.orm import (
    Mapped,
    mapped_column,
    DeclarativeBase,
    relationship,
    sessionmaker,
)


class Base(DeclarativeBase):
    pass


class User(Base):
    __tablename__ = "user_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    groups: Mapped[List[Group]] = relationship(
        back_populates="users", secondary="link_table"
    )
    links: Mapped[List[Link]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"< {self.__class__.__name__} {self.id} >"


class Group(Base):
    __tablename__ = "group_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    users: Mapped[List[User]] = relationship(
        back_populates="groups", secondary="link_table"
    )
    links: Mapped[List[Link]] = relationship(back_populates="group")

    def __repr__(self) -> str:
        return f"< {self.__class__.__name__} {self.id} >"


class Link(Base):
    __tablename__ = "link_table"

    user_id: Mapped[int] = mapped_column(ForeignKey("user_table.id"), primary_key=True)
    group_id: Mapped[int] = mapped_column(
        ForeignKey("group_table.id"), primary_key=True
    )

    user: Mapped[User] = relationship(back_populates="links")
    group: Mapped[Group] = relationship(back_populates="links")

    def __repr__(self) -> str:
        return f"< User {self.user_id} - Group {self.group_id} >"


engine = create_engine("sqlite://", echo=False)
session_factory = sessionmaker(bind=engine)

with session_factory() as session:
    Base.metadata.create_all(engine)

    # ------------------------------------
    # First test
    # ------------------------------------

    u1 = User()
    g1 = Group()
    g2 = Group()
    u1.groups.extend([g1, g2])

    session.add_all([u1, g1, g2])
    session.commit()

    assert u1.groups == [g1, g2]
    assert g1.users == [u1]
    assert g2.users == [u1]
    assert len(u1.links) == 2

/tmp/ipykernel_24363/783346028.py:69: SAWarning: relationship 'User.links' will copy column user_table.id to column link_table.user_id, which conflicts with relationship(s): 'User.groups' (copies user_table.id to link_table.user_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="groups"' to the 'User.links' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  u1 = User()
/tmp/ipykernel_24363/783346028.py:69: SAWarning: relationship 'Group.users' will copy column use

### Lazy vs. Eager Relationships

| Loading Strategy | Type          | Description                                                                                                         | Function                     |
|-------------------|---------------|---------------------------------------------------------------------------------------------------------------------|------------------------------|
| **Lazy Load**     | `select`      | Default behavior. Loads related data only when accessed. Issues a new SELECT query for each access.                 | (Default)                   |
| **Eager Load**    | `joined`      | Loads related data in the same query using a SQL JOIN. Reduces the number of queries but increases query complexity. | `joinedload()`              |
|                   | `selectin`    | Loads related data with additional SELECT queries but batches them for efficiency.                                  | `selectinload()`            |
|                   | `subquery`    | Similar to `joined` but uses a subquery for loading related data.                                                   | `subqueryload()`            |
|                   | `immediate`   | Loads related data immediately after the parent object is loaded. Useful for small related datasets.                | `immediateload()`           |
| **Explicit Load** | `write_only`  | Disables reading of related data but allows writing to the relationship.                                            | `write_only` (config option)|
|                   | `noload`      | Does not load related data even when accessed. Returns `None` unless explicitly populated.                         | `noload()`                  |
|                   | `raise`       | Raises an exception if the related attribute is accessed, ensuring explicit handling of relationships.              | `raiseload()`               |
|                   | `raise_on_sql`| Similar to `raise` but raises an exception only if an implicit SQL query would be triggered.                        | `raiseload()` (with options)|



### Cascade configurations

Default is `save update, merge`

| Cascade Configuration   | Description                                                                                               | Typical Use Case                                     |
|--------------------------|-----------------------------------------------------------------------------------------------------------|-----------------------------------------------------|
| `save-update`            | Automatically saves or updates related objects when the parent is saved or updated.                      | Keep parent and related objects in sync during persistence. |
| `merge`                  | Merges changes in a related object into the session when the parent is merged.                           | Used when copying objects or working across sessions.       |
| `delete`                 | Deletes related objects when the parent is deleted.                                                      | Ensure related objects are removed alongside the parent.     |
| `delete-orphan`          | Deletes related objects if they are no longer associated with the parent.                                | Maintain strict ownership, removing unreferenced objects.    |
| `expunge`                | Removes related objects from the session when the parent is removed.                                     | Useful when cleaning up detached objects.                   |
| `refresh-expire`         | Automatically refreshes or expires related objects when the parent is refreshed or expired.              | Ensure related objects stay consistent with the parent.      |
| `all`                    | Applies all cascade behaviors except `delete-orphan` (`save-update`, `merge`, `delete`, `refresh-expire`).| Use for general cascading without orphan cleanup.            |              |
| `none`                   | No cascade behavior is applied.                                                                          | For relationships that require explicit handling.            |
